In [0]:
from pyspark.sql.functions import col, count, desc, lit, when, datediff, sum, avg, regexp_replace, to_date, to_timestamp, hour, struct, collect_list, to_json
import requests
import json
from utils import spark_df_to_json_serializable
from datetime import datetime
import pytz

In [0]:
df_pedidos_ = spark.read.csv(
    "/Volumes/transacional/case_gocase/tmp/Case Dados - Pedidos.csv",
    header=True,
    inferSchema=True
)
df_itens = spark.read.csv(
    "/Volumes/transacional/case_gocase/tmp/Business Case Dados Itens.csv",
    header=True,
    inferSchema=True
)
df_supply = spark.read.csv(
    "/Volumes/transacional/case_gocase/tmp/Business Case Dados Supply.csv",
    header=True,
    inferSchema=True
)


In [0]:
# 1. Renomear colunas 
df_pedidos = (df_pedidos_
    .withColumnRenamed("CÃ³digo de Rastreio", "codigo_rastreio")
    .withColumnRenamed("Valor de NF (R$)", "valor_nf")
    .withColumnRenamed("Frete Cobrado do Cliente (R$)", "frete_cliente")
    .withColumnRenamed("Frete cobrado pela transportadora (R$)", "frete_transportadora")
    .withColumnRenamed("NÃºmero da NF", "numero_nf")
    .withColumnRenamed("Status do Pedido", "status_pedido")
    .withColumnRenamed("Prazo para Sair do CD", "prazo_saida_cd")
    .withColumnRenamed("Enviado em:", "data_envio")
    .withColumnRenamed("Entregue para o cliente em:", "data_entrega")
    .withColumnRenamed("Prazo a transportadora entregar no cliente", "prazo_entrega_transportadora")
    .withColumnRenamed("NÃºmero de Itens no Pedido", "num_itens")
    .withColumnRenamed("Peso (kg)", "peso_kg")
    .withColumnRenamed("id", "order_id")
)

# 2. Converter colunas numéricas
cols_to_convert_pedidos = ["valor_nf", "frete_cliente", "frete_transportadora", "peso_kg"]
for column in cols_to_convert_pedidos:
    df_pedidos = df_pedidos.withColumn(column, regexp_replace(regexp_replace(col(column), "\\.", ""), ",", ".").cast("double"))

cols_to_convert_itens = ["material_weight_kg", "price"]
for column in cols_to_convert_itens:
    df_itens = df_itens.withColumn(column, regexp_replace(regexp_replace(col(column), "\\.", ""), ",", ".").cast("double"))

df_supply = df_supply.withColumn("quantity", regexp_replace(regexp_replace(col("quantity"), "\\.", ""), ",", ".").cast("double"))



# Dicionário para "traduzir" os meses, será usado para ambos os formatos
month_map = {
    "jan.,": "Jan", "fev.,": "Feb", "mar.,": "Mar", "abr.,": "Apr",
    "mai.,": "May", "jun.,": "Jun", "jul.,": "Jul", "ago.,": "Aug",
    "set.,": "Sep", "out.,": "Oct", "nov.,": "Nov", "dez.,": "Dec"
}

# 3.1 Tratar colunas que SÓ TÊM DATA
date_only_cols = ["prazo_saida_cd", "data_envio", "data_entrega", "prazo_entrega_transportadora"]
format_date_only = "d MMM yyyy"

for date_col in date_only_cols:
    expr = col(date_col)
    for pt_month, en_month in month_map.items():
        expr = regexp_replace(expr, pt_month, en_month) 
    df_pedidos = df_pedidos.withColumn(date_col, to_date(expr, format_date_only))

# 3.2: Tratar a coluna `created_at` (formato "28 fev., 2025, 23:42")
format_datetime = "d MMM yyyy, HH:mm"
expr_ts = col("created_at")

for pt_month, en_month in month_map.items():
    expr_ts = regexp_replace(expr_ts, pt_month, en_month)

# Aplica as transformações em cadeia para criar as novas colunas
df_pedidos = (df_pedidos
    # 1. Cria uma coluna temporária com o timestamp completo
    .withColumn("created_at_ts", to_timestamp(expr_ts, format_datetime))
    # 2. Cria a coluna de DATA a partir do timestamp
    .withColumn("created_at_data", to_date(col("created_at_ts")))
    # 3. Cria a coluna de HORA a partir do timestamp
    .withColumn("created_at_hour", hour(col("created_at_ts")))
    # 4. Remove a coluna original e a temporária
    .drop("created_at", "created_at_ts")
)


# 4. Criar DataFrame base para análise
df_vendas = df_pedidos.join(df_itens, "order_id", "inner")


In [0]:
#  1.1. Análise de Vendas ao Longo do Tempo
# Vamos visualizar o faturamento e o número de pedidos por dia para identificar picos, quedas e padrões.
# O que analisar nos resultados:
# Picos de Vendas: Existem dias específicos (finais de semana, início/fim de mês) com vendas muito mais altas?
# Sazonalidade: Você consegue observar algum padrão semanal ou mensal?

vendas_diarias = df_pedidos.groupBy("created_at_data") \
                           .agg(
                               sum("valor_nf").alias("faturamento_total"),
                               count("order_id").alias("numero_pedidos")
                           ).orderBy("created_at_data")

print("--- Vendas Diárias (Faturamento e N° de Pedidos) ---")
display(vendas_diarias)

Databricks visualization. Run in Databricks to view.

In [0]:
# 1.2. Impacto da Promoção de Frete Grátis
# Queremos saber se oferecer frete grátis incentiva os clientes a gastarem mais ou a comprarem mais itens.
# O que analisar nos resultados:
# Ticket Médio: O valor médio do pedido (ticket_medio) é significativamente maior nos pedidos com "Frete Grátis"? Se sim, a promoção está se pagando.
# Média de Itens: Os clientes adicionam mais produtos ao carrinho para atingir o frete grátis? Compare a media_itens_por_pedido.
# Segmentamos os pedidos em dois grupos: com frete pago e com frete grátis.

analise_frete = (df_pedidos.withColumn("tipo_frete", when(col("frete_cliente") > 0, "Pago")
                           .otherwise("Grátis"))
                           .groupBy("tipo_frete") 
                           .agg(avg("valor_nf").alias("ticket_medio"),
                                avg("num_itens").alias("media_itens_por_pedido"),
                                count("order_id").alias("total_pedidos")
                                )
                           )
analise_frete_dia = (df_pedidos.withColumn("tipo_frete", when(col("frete_cliente") > 0, "Pago")
                           .otherwise("Grátis"))
                           .groupBy("tipo_frete","created_at_data") 
                           .agg(avg("valor_nf").alias("ticket_medio"),
                                avg("num_itens").alias("media_itens_por_pedido"),
                                count("order_id").alias("total_pedidos")
                                )
                           )
analise_frete_dia_gratis=(analise_frete_dia.select("created_at_data", "media_itens_por_pedido")
                                           .filter(col("tipo_frete")=="Grátis")
                                           .orderBy("created_at_data")
                                           )
analise_frete_dia_pago=(analise_frete_dia.select("created_at_data", "media_itens_por_pedido")
                                           .filter(col("tipo_frete")=="Pago")
                                           .orderBy("created_at_data")
                                           )

print("--- Análise de Impacto do Frete Grátis ---")
display(analise_frete)
display(analise_frete_dia_gratis)
display(analise_frete_dia_pago)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
# 1.3 Quais categorias ou produtos possuem maior impacto no faturamento?
# --- Ranking de Faturamento por Categoria ---

faturamento_por_categoria = df_vendas.groupBy("material_category") \
                                     .agg(sum("price").cast("integer").alias("faturamento_total"),
                                         count("order_id").alias("unidades_vendidas")
                                     ).orderBy(desc("faturamento_total"))

print("--- Ranking de Faturamento por Categoria de Produto ---")
display(faturamento_por_categoria)

# --- Ranking de Faturamento por Produto ---

faturamento_por_produto = df_vendas.groupBy("material_name") \
                                   .agg(sum("price").cast("integer").alias("faturamento_total"),
                                       count("order_id").alias("unidades_vendidas")
                                   ).orderBy(desc("faturamento_total"))
faturamento_por_produto_top20 = faturamento_por_produto.limit(20)
print("\n--- Top 20 Produtos por Faturamento (como inteiro) ---")
display(faturamento_por_produto_top20)

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
# Análise do Desafio 2: Rupturas de Estoque Objetivo: Identificar quais produtos estão em falta e, mais importante, qual o impacto potencial disso na receita.
#  2.1. Identificar Produtos Críticos em Ruptura
#  --- 1. Sua consulta original para identificar os produtos em ruptura ---
# Primeiro, listamos os produtos que deveriam estar à venda, mas estão com estoque zerado.
produtos_ruptura = df_supply.filter(
    (col("quantity") == 0) &
    (col("should_sell") == True) &
    (col("discontinued") == False)
).select("material_id", "material_name", "leadtime")

print("--- Amostra de Produtos em Ruptura de Estoque ---")
display(produtos_ruptura.limit(10))

produtos_ruptura_unicos = produtos_ruptura.distinct()

# --- 2. NOVO CÓDIGO: Cálculo do percentual ---

# Contar o número total de itens ÚNICOS no catálogo (baseado no df_supply)
total_itens_unicos = df_supply.select("material_id").distinct().count()

# Contar o número de itens ÚNICOS que estão em ruptura (baseado no df produtos_ruptura)
itens_unicos_em_ruptura = produtos_ruptura.select("material_id").distinct().count()

# Calcular o percentual e exibir os resultados de forma clara
print("\n--- Análise do Percentual de Ruptura de Estoque ---")
print(f"Total de produtos únicos no catálogo: {total_itens_unicos}")
print(f"Total de produtos únicos em ruptura: {itens_unicos_em_ruptura}")

percentual_ruptura = (itens_unicos_em_ruptura / total_itens_unicos) * 100
print(f"Percentual de itens do catálogo em ruptura: {percentual_ruptura:.2f}%")


In [0]:
# 2.2. Priorizar a Reposição: Cruzando Ruptura com Demanda
# Um produto em ruptura só é um problema se ele vende bem. Vamos cruzar a lista de ruptura com o histórico de vendas para encontrar os casos mais críticos.
# O que analisar nos resultados:

# Lista de Prioridades: O topo desta tabela (ruptura_critica) são os produtos que você precisa repor com mais urgência. Eles estão em falta e têm uma alta demanda histórica.

# leadtime vs. vendas_historicas: Um produto com vendas altíssimas e um leadtime (tempo de reposição) longo é um grande risco para a receita.
# Contar as vendas históricas para cada material_id
demanda_historica = df_vendas.groupBy("material_id") \
                             .agg(count("order_id").alias("vendas_historicas"))

# Juntar a lista de ruptura com a demanda para priorizar
ruptura_critica = (produtos_ruptura_unicos.join(demanda_historica, "material_id", "inner") 
                                   .orderBy(col("vendas_historicas").desc())                                   
                                   )
ruptura_critica_top20 = ruptura_critica.limit(20)
print("--- Produtos Mais Críticos para Reposição (Ruptura vs. Demanda) ---")
display(ruptura_critica_top20)

# Juntar a lista de ruptura com o faturamento por produto
ruptura_critica_faturamento_top20 = (ruptura_critica.join(faturamento_por_produto, "material_name", "inner") 
                                              .orderBy(col("faturamento_total").desc())
                                              .limit(20)
                                              )

print("\n--- Produtos Mais Críticos para Reposição (Ruptura vs. Faturamento) ---")    
display(ruptura_critica_faturamento_top20)


Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

In [0]:
# Análise do Desafio 3: Problemas Logísticos Objetivo: Medir a eficiência da entrega, identificar atrasos e comparar o desempenho de transportadoras e regiões
#  3.1. Calcular Tempos do Ciclo de Entrega
# Vamos medir o tempo que um pedido leva para sair do armazém e o tempo que a transportadora leva para entregá-lo.



# Calculamos a diferença em dias entre as etapas do pedido
df_pedidos_com_prazos = df_pedidos.withColumn("tempo_despacho_dias", datediff(col("data_envio"), col("created_at_data"))) \
                                 .withColumn("tempo_transporte_dias", datediff(col("data_entrega"), col("data_envio")))

# Calcular médias gerais
medias_logisticas = df_pedidos_com_prazos.agg(
    avg("tempo_despacho_dias").alias("media_dias_para_despacho"),
    avg("tempo_transporte_dias").alias("media_dias_em_transporte")
)

print("--- Médias de Tempo do Ciclo Logístico ---")
display(medias_logisticas)

In [0]:
# 3.2. Análise de Atrasos e Desempenho por Transportadora
# Estamos cumprindo os prazos? Qual transportadora é mais rápida e qual tem mais atrasos?
# O que analisar nos resultados:

# Ranking de Transportadoras: Qual transportadora tem o menor tempo_medio_entrega e o menor percentual_atrasos? Isso pode ser crucial para renegociar contratos.

# Análise por Estado: Você pode facilmente adaptar a consulta acima para fazer um groupBy("Estado") em vez de groupBy("Transportadora") e descobrir quais estados têm os maiores prazos ou taxas de atraso.
# Criar uma flag para pedidos atrasados
df_pedidos_com_atraso = df_pedidos_com_prazos.withColumn(
    "atrasado",
    when(col("data_entrega") > col("prazo_entrega_transportadora"), 1).otherwise(0)
)

# Agrupar por transportadora para analisar o desempenho
desempenho_transportadora = df_pedidos_com_atraso.groupBy("Transportadora") \
    .agg(
        avg("tempo_transporte_dias").alias("tempo_medio_entrega"),
        (sum("atrasado") / count("order_id") * 100).alias("percentual_atrasos"),
        count("order_id").alias("total_entregas")
    ).orderBy(col("tempo_medio_entrega"))

print("--- Desempenho por Transportadora ---")
display(desempenho_transportadora)

Databricks visualization. Run in Databricks to view.

In [0]:

# Análise 1.1: Oscilações nas Vendas e Impacto de Promoções
vendas_diarias_resultado = spark_df_to_json_serializable(vendas_diarias, limit=31)

# Análise 1.2: Impacto Frete Grátis
analise_frete_resultado = spark_df_to_json_serializable(analise_frete, limit=10)

# Análise 1.2: Impacto Frete Grátis dia
analise_frete_dia_gratis_resultado = spark_df_to_json_serializable(analise_frete_dia_gratis, limit=31)
analise_frete_dia_pago_resultado = spark_df_to_json_serializable(analise_frete_dia_pago, limit=31)

# Análise 1.3: Quais categorias ou produtos possuem maior impacto no faturamento?
faturamento_por_categoria_resultado = spark_df_to_json_serializable(faturamento_por_categoria, limit=50)
faturamento_por_produto_resultado = spark_df_to_json_serializable(faturamento_por_produto_top20, limit=50)

# Análise 2.1: Identificar Produtos Críticos em Ruptura
total_itens_unicos_resultado = total_itens_unicos
itens_unicos_em_ruptura_resultado = itens_unicos_em_ruptura
percentual_ruptura_resultado = percentual_ruptura

# Análise 2.2: Priorizar a Reposição: Cruzando Ruptura com Demanda
ruptura_critica_resultado = spark_df_to_json_serializable(ruptura_critica_top20, limit=20)
ruptura_critica_faturamento_resultado = spark_df_to_json_serializable(ruptura_critica_faturamento_top20, limit=20)

# Análise 3.1: Calcular Tempos do Ciclo de Entrega
medias_logisticas_resultado = spark_df_to_json_serializable(medias_logisticas, limit=10)

# Analise 3.2: Análise de Atrasos e Desempenho por Transportadora
desempenho_transportadoras_resultado = spark_df_to_json_serializable(desempenho_transportadora, limit=10)

# Define o fuso horário de Brasília (UTC-3)
fuso_horario_brasil = pytz.timezone('America/Sao_Paulo') 
# Obtém a data e hora atuais nesse fuso horário
agora_em_brasilia = datetime.now(fuso_horario_brasil)
# Formata a data e hora em uma string no formato desejado
timestamp_formatado = agora_em_brasilia.strftime('%d/%m/%Y %H:%M')

# --- 2. Crie o "pacote" de dados (dicionário mestre) ---
# Cada chave do dicionário representa um relatório diferente.
master_payload = {
    "relatorio_vendas_diarias": vendas_diarias_resultado,
    "relatorio_analise_fretes": analise_frete_resultado,
    "relatorio_analise_fretes_dia_gratis": analise_frete_dia_gratis_resultado,
    "relatorio_analise_fretes_dia_pago": analise_frete_dia_pago_resultado,
    "faturamento_por_categoria_resultado": faturamento_por_categoria_resultado,
    "faturamento_por_produto_resultado": faturamento_por_produto_resultado,
    "total_itens_unicos_resultado": total_itens_unicos_resultado,
    "itens_unicos_em_ruptura_resultado": itens_unicos_em_ruptura_resultado,
    "percentual_ruptura_resultado": percentual_ruptura_resultado,
    "ruptura_critica_resultado": ruptura_critica_resultado,
    "ruptura_critica_faturamento_resultado": ruptura_critica_faturamento_resultado,
    "medias_logisticas_resultado": medias_logisticas_resultado,
    "desempenho_transportadoras_resultado": desempenho_transportadoras_resultado,
    "generation_timestamp": timestamp_formatado
}

# --- 3. Envie o pacote completo para o n8n ---
#n8n_webhook_url = "https://emanoeljavier.app.n8n.cloud/webhook/2c0caced-9c0d-40b0-838f-04daab4f62ca"
n8n_webhook_url ="https://emanoeljavier.app.n8n.cloud/webhook-test/2c0caced-9c0d-40b0-838f-04daab4f62ca" #test
headers = {'Content-Type': 'application/json'}

try:
    print("\nEnviando pacote de dados completo para o n8n...")
    response = requests.post(n8n_webhook_url, data=json.dumps(master_payload), headers=headers)
    response.raise_for_status()
    print("✅ Pacote de dados enviado com sucesso para o n8n!")
    print(f"Resposta do n8n: {response.json()}")

except Exception as e:
    print(f"❌ Falha ao enviar dados para o n8n: {e}")